In [1]:
%%capture
!pip install webdriver-manager
!pip install selenium pandas 
!pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from pprint import pprint
import random
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime
import re

In [2]:
def load_cookies(driver, cookie_file):
    try:
        with open(cookie_file, "r") as file:
            cookies = json.load(file)
            if not cookies:
                print("⚠️ Cookie file is empty!")
                return
            
            for cookie in cookies:
                # Convert ISO 8601 expiry to UNIX timestamp
                if "expiry" in cookie:
                    try:
                        cookie["expiry"] = int(datetime.strptime(cookie["expiry"], "%Y-%m-%dT%H:%M:%S.%fZ").timestamp())
                    except ValueError:
                        print(f"⚠️ Skipping cookie {cookie['name']} due to invalid expiry format: {cookie['expiry']}")
                        del cookie["expiry"]  # Remove invalid expiry field
                
                driver.add_cookie(cookie)
            print("✅ Cookies loaded successfully!")
    except json.JSONDecodeError:
        print("⚠️ Error: Invalid JSON format in the cookie file.")
    except Exception as e:
        print(f"⚠️ Error while loading cookies: {e}")

In [3]:
def convert_to_number(value):
    """Convertit les nombres abrégés de Twitter (e.g., '1K', '2.5M') en entiers."""
    value = value.replace(',', '').strip().upper()
    if 'K' in value:
        return int(float(value.replace('K', '')) * 1_000)
    elif 'M' in value:
        return int(float(value.replace('M', '')) * 1_000_000)
    elif 'B' in value:
        return int(float(value.replace('B', '')) * 1_000_000_000)
    else:
        try:
            return int(value)
        except ValueError:
            return 0  # Retourne 0 si la conversion échoue

In [4]:
def extract_tweet_data(tweet):
    """Extract detailed information from a tweet element."""
    try:
        user = tweet.find_element(By.CSS_SELECTOR, 'div[dir="ltr"] span').text
    except:
        user = "Unknown"

    try:
        content = tweet.text.replace("\n", " ")
    except:
        content = ""

    try:
        date = tweet.find_element(By.CSS_SELECTOR, 'time').get_attribute("datetime")
    except:
        date = "Unknown"

    # Extraction des valeurs de Likes, Retweets et Replies depuis la fin du tweet
    numbers = re.findall(r"\d[\d,\.]*[KM]?", content)  # Récupère les nombres du texte
    likes, retweets, replies, views = "0", "0", "0", "0"
    
    if len(numbers) >= 4:
        replies, retweets, likes, views = numbers[-4:]
        
    replies = convert_to_number(replies)
    retweets = convert_to_number(retweets)
    likes = convert_to_number(likes)
    views = convert_to_number(views)


    # Nettoyage du contenu 
    content_cleaned = re.sub(r"\d[\d,\.]*[KM]?$", "", content).strip()
    content_cleaned = re.sub(r"\d[\d,\.]*[KM]?$", "", content_cleaned).strip()
    content_cleaned = re.sub(r"\d[\d,\.]*[KM]?$", "", content_cleaned).strip()
    content_cleaned = re.sub(r"\d[\d,\.]*[KM]?$", "", content_cleaned).strip()

    # Extraction des hashtags et mentions
    hashtags = ", ".join(re.findall(r"#\w+", content_cleaned))
    mentions = ", ".join(re.findall(r"@\w+", content_cleaned))
    # timestamps = ", ".join(re.findall(r"(?<=·\s)\d+\w*", content_cleaned))

       
    content_cleaned = re.sub(r'^.*?·\s*(\d+\w*|\w+\s+\d{1,2}(?:,\s*\d{4})?)\s*', '', content_cleaned)


    return {
        "User": user,
        "Date": date,
        # "Time" : timestamps,
        "Content": content_cleaned,
        "Hashtags": hashtags,
        "Mentions": mentions,
        "Views" : views,
        "Likes": likes,
        "Retweets": retweets,
        "Replies": replies
    }

In [5]:
# Function to fetch trending tweets
def fetch_trending_tweets(cookie_file="cookies.json", max_tweets=500):
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36")
    
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.get("https://x.com/home")
    time.sleep(5)

    # Load cookies and refresh
    load_cookies(driver, cookie_file)
    driver.refresh()
    time.sleep(5)

    print("Current page title:", driver.title)  # Debugging login state

    # Verify if cookies are still valid
    print("Final Cookies in Browser:", driver.get_cookies())

    # Go to trending page
    search_query = "breaking OR viral OR trending OR news"
    search_url = f"https://x.com/search?q={search_query.replace(' ', '%20')}&src=typed_query&f=top"
    driver.get(search_url)
    # driver.get("https://x.com/search?q=trending&src=trend_click")
    time.sleep(5)

    tweet_data = []
    tweet_count = 0

    while tweet_count < max_tweets:
        tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')
        print(f"Found {len(tweets)} tweets on this page.")

        for tweet in tweets:
            data = extract_tweet_data(tweet)  # Extract structured tweet data
            tweet_data.append(data)  # Append only the structured data
            tweet_count += 1
            if tweet_count >= max_tweets:
                break

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

    driver.quit()
    return tweet_data

In [6]:
# Run the scraper
tweets = fetch_trending_tweets()

✅ Cookies loaded successfully!
Current page title: Home / X
Final Cookies in Browser: [{'domain': '.x.com', 'expiry': 1771013134, 'httpOnly': False, 'name': 'twid', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': 'u%3D1825171960537513984'}, {'domain': 'x.com', 'httpOnly': False, 'name': 'lang', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'en'}, {'domain': '.x.com', 'expiry': 1773924258, 'httpOnly': False, 'name': 'ct0', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': 'f86986a4ce28c953c89d10767b9e39adf910b9c31489cfca3f8c793fb7152cb215ebb1050abf53db9904e1a4b447b9c8b195bcfd9670d0159cf157c84c7336f5768b390248cc8b0be4d9dfb515163407'}, {'domain': '.x.com', 'expiry': 1773924257, 'httpOnly': True, 'name': 'auth_token', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': '4b37f027aa860025c02899f572d0cb6994f2888f'}, {'domain': '.x.com', 'expiry': 1774037134, 'httpOnly': False, 'name': 'guest_id_ads', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': 

In [7]:
import csv

def save_tweets_to_csv(tweets, output_csv="tweets.csv"):
    if not tweets:
        print("⚠️ No tweets to save!")
        return
    
    fieldnames = ["User", "Date", "Content", "Hashtags", "Mentions", "Views", "Likes", "Retweets", "Replies"]

    with open(output_csv, "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(tweets)

    print(f"✅ {len(tweets)} tweets saved to {output_csv} with structured data!")

In [8]:
# Save the structured tweets
save_tweets_to_csv(tweets)

✅ 500 tweets saved to tweets.csv with structured data!


In [9]:
tweets = pd.read_csv('tweets.csv')

In [10]:
tweets

,User,Date,Content,Hashtags,Mentions,Views,Likes,Retweets,Replies
0,Really American,2025-02-13T19:22:12.000Z,BREAKING: Donald Trump just announced reciproc...,NaN,@ReallyAmerican1,29000,1400,543,128
1,DogeDesigner,2025-02-13T18:44:36.000Z,BREAKING: 𝕏 added a new like animation for the...,#ValentinesDay,@cb_doge,54000,2500,225,282
2,xxx bokep,2025-02-13T19:53:43.000Z,Terbaru!! Bulan Sutena 1 Menit 14 Detik bulan ...,NaN,@CarolinaDynasty,1900,101,8,92
3,Matt Van Swol,2025-02-13T16:42:31.000Z,#BREAKING: The state of TN has just ELIMINATED...,#BREAKING,@matt_vanswol,330000,35000,7600,904
4,MJTruthUltra,2025-02-13T20:00:40.000Z,BREAKING: President Trump says Canada is a ver...,NaN,@MJTruthUltra,3200,114,28,12
...,...,...,...,...,...,...,...,...,...
495,Oxomiya Jiyori,2025-02-13T12:36:43.000Z,BREAKING: A driver plowed a Mini cooper into a...,NaN,@SouleFacts,16000,350,173,24
496,Nigeria Stories,2025-02-13T15:00:30.000Z,BREAKING NEWS: Nigeria National Assembly pass...,NaN,@NigeriaStories,30000,749,105,48
497,Pi News,2025-02-13T10:41:00.000Z,$PI is ranked 3rd on CMC Hot List!,NaN,@PiNewsApp,65000,1300,152,150
498,New York Post,2025-02-13T14:08:31.000Z,Telemundo reporter Adam Mananzo seen just befo...,NaN,@nypost,46000,189,54,43


In [11]:
duplicated_content_values = tweets[tweets['Content'].duplicated(keep=False)]['Content']

In [12]:
duplicated_content_values.shape

(198,)

In [13]:
df_unique_content = tweets.drop_duplicates(subset=['Content'], keep='first')

In [14]:
df_unique_content.shape

(334, 9)

In [15]:
df_unique_content

,User,Date,Content,Hashtags,Mentions,Views,Likes,Retweets,Replies
0,Really American,2025-02-13T19:22:12.000Z,BREAKING: Donald Trump just announced reciproc...,NaN,@ReallyAmerican1,29000,1400,543,128
1,DogeDesigner,2025-02-13T18:44:36.000Z,BREAKING: 𝕏 added a new like animation for the...,#ValentinesDay,@cb_doge,54000,2500,225,282
2,xxx bokep,2025-02-13T19:53:43.000Z,Terbaru!! Bulan Sutena 1 Menit 14 Detik bulan ...,NaN,@CarolinaDynasty,1900,101,8,92
3,Matt Van Swol,2025-02-13T16:42:31.000Z,#BREAKING: The state of TN has just ELIMINATED...,#BREAKING,@matt_vanswol,330000,35000,7600,904
4,MJTruthUltra,2025-02-13T20:00:40.000Z,BREAKING: President Trump says Canada is a ver...,NaN,@MJTruthUltra,3200,114,28,12
...,...,...,...,...,...,...,...,...,...
357,Pi News,2025-02-13T10:41:00.000Z,$PI is ranked 3rd on CMC Hot List!,NaN,@PiNewsApp,64000,1300,152,150
358,New York Post,2025-02-13T14:08:31.000Z,Telemundo reporter Adam Mananzo seen just befo...,NaN,@nypost,46000,187,53,43
359,Oxomiya Jiyori,2025-02-13T12:36:43.000Z,BREAKING: A driver plowed a Mini cooper into a...,NaN,@SouleFacts,16000,350,173,24
360,Nigeria Stories,2025-02-13T15:00:30.000Z,BREAKING NEWS: Nigeria National Assembly pass...,NaN,@NigeriaStories,30000,747,104,48


In [16]:
df_unique_content.to_csv('tweets.csv', index=False)

In [17]:
tweets = pd.read_csv('tweets.csv')

In [18]:
tweets

,User,Date,Content,Hashtags,Mentions,Views,Likes,Retweets,Replies
0,Really American,2025-02-13T19:22:12.000Z,BREAKING: Donald Trump just announced reciproc...,NaN,@ReallyAmerican1,29000,1400,543,128
1,DogeDesigner,2025-02-13T18:44:36.000Z,BREAKING: 𝕏 added a new like animation for the...,#ValentinesDay,@cb_doge,54000,2500,225,282
2,xxx bokep,2025-02-13T19:53:43.000Z,Terbaru!! Bulan Sutena 1 Menit 14 Detik bulan ...,NaN,@CarolinaDynasty,1900,101,8,92
3,Matt Van Swol,2025-02-13T16:42:31.000Z,#BREAKING: The state of TN has just ELIMINATED...,#BREAKING,@matt_vanswol,330000,35000,7600,904
4,MJTruthUltra,2025-02-13T20:00:40.000Z,BREAKING: President Trump says Canada is a ver...,NaN,@MJTruthUltra,3200,114,28,12
...,...,...,...,...,...,...,...,...,...
329,Pi News,2025-02-13T10:41:00.000Z,$PI is ranked 3rd on CMC Hot List!,NaN,@PiNewsApp,64000,1300,152,150
330,New York Post,2025-02-13T14:08:31.000Z,Telemundo reporter Adam Mananzo seen just befo...,NaN,@nypost,46000,187,53,43
331,Oxomiya Jiyori,2025-02-13T12:36:43.000Z,BREAKING: A driver plowed a Mini cooper into a...,NaN,@SouleFacts,16000,350,173,24
332,Nigeria Stories,2025-02-13T15:00:30.000Z,BREAKING NEWS: Nigeria National Assembly pass...,NaN,@NigeriaStories,30000,747,104,48
